<a href="https://colab.research.google.com/github/PraeJi/Brain-Tumor-Detection/blob/main/brain_tumor_detection_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Note*** : Brain tumor detection in this version made folder augmented_images and delete folder images in train then rename folder augmented_images to images

In [ ]:
!nvidia-smi

Mon Sep 16 02:45:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load Dataset

In [ ]:
!kaggle datasets download -d pkdarabi/medical-image-dataset-brain-tumor-detection

Dataset URL: https://www.kaggle.com/datasets/pkdarabi/medical-image-dataset-brain-tumor-detection
License(s): Attribution 4.0 International (CC BY 4.0)
 93% 105M/113M [00:02<00:00, 66.8MB/s] 
100% 113M/113M [00:02<00:00, 49.6MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile('/content/medical-image-dataset-brain-tumor-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

# Install and Import Library

In [ ]:
# Install Essential Libraries
!pip install ultralytics
from IPython import display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
# Import Essential Libraries
import os
import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Create Dataframe

In [ ]:
import os
from os import listdir

folder_test_img = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/test/images"
folder_test_label = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/test/labels"

folder_train_img = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images"
folder_train_label = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/labels"

folder_val_img = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/valid/images"
folder_val_label = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/valid/labels"

In [ ]:
data_train = []
data_test = []
data_val = []

## Calculate xmin,ymin,xmax,ymax Function

In [ ]:
# Calculate xmin,ymin,xmax,ymax

def calculate(file_path,filename,x_center,y_center,width,height):
  path_img = f"{file_path}/{filename}"
  im = Image.open(path_img).convert("RGB")
  img_width, img_height = im.size

  x_center_pixel = x_center * img_width
  y_center_pixel = y_center * img_height
  half_width_pixel = width * img_width / 2
  half_height_pixel = height * img_height / 2

  x_min = x_center_pixel - half_width_pixel
  y_min = y_center_pixel - half_height_pixel
  x_max = x_center_pixel + half_width_pixel
  y_max = y_center_pixel + half_height_pixel

  return x_min,y_min,x_max,y_max

## Test Dataframe

In [ ]:
for labels_file in os.listdir(folder_test_label):
  file_path = f"{folder_test_label}/{labels_file}" #.txt
  file = open(file_path,"r")
  content = file.readlines()
  filename = labels_file[:-4] + '.jpg'

  for line in content:
    parts = line.strip().split()
    label = int(parts[0])
    x_center = float(parts[1])
    y_center = float(parts[2])
    width = float(parts[3])
    height = float(parts[4])

    x_min,y_min,x_max,y_max = calculate(folder_test_img, filename, x_center, y_center, width,height)

    data_test.append({
      'Filename': filename,
      'Label': int(label),
      'x_Center': x_center,
      'y_Center': y_center,
      'width': width,
      'height': height,
      'x_min': x_min,
      'y_min': y_min,
      'x_max': x_max,
      'y_max': y_max,
      #'path' : f"{folder_test_img}/"
    })

  file.close

In [ ]:
df_test = pd.DataFrame(data_test)
#df_test['Filename'] = df_test['Filename'] + '.jpg'
df_test.head()

,Filename,Label,x_Center,y_Center,width,height,x_min,y_min,x_max,y_max
0,volume_7_slice_49_jpg.rf.6b5dea062160d2d435a4d...,0,0.654676,0.844697,0.100719,0.113636,84.0,104.0,98.0,119.0
1,volume_7_slice_49_jpg.rf.6b5dea062160d2d435a4d...,1,0.607914,0.844697,0.395683,0.295455,57.0,92.0,112.0,131.0
2,volume_7_slice_49_jpg.rf.6b5dea062160d2d435a4d...,2,0.647482,0.844697,0.172662,0.143939,78.0,102.0,102.0,121.0
3,volume_35_slice_47_jpg.rf.348a0bfb80e6e580313e...,0,0.420863,0.219697,0.395683,0.303030,31.0,9.0,86.0,49.0
4,volume_35_slice_47_jpg.rf.348a0bfb80e6e580313e...,1,0.514388,0.181818,0.482014,0.363636,38.0,0.0,105.0,48.0


In [ ]:
df_test.to_csv('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/test/test.csv')

## Train Dataframe

In [ ]:
for labels_file in os.listdir(folder_train_label):
  file_path = f"{folder_train_label}/{labels_file}"
  file = open(file_path,"r")
  content = file.readlines()
  filename = labels_file[:-4] + '.jpg'

  for line in content:
    parts = line.strip().split()
    label = int(parts[0])
    x_center = float(parts[1])
    y_center = float(parts[2])
    width = float(parts[3])
    height = float(parts[4])

    x_min,y_min,x_max,y_max = calculate(folder_train_img,filename,x_center,y_center,width,height)

    data_train.append({
      'Filename': filename,
      'Label': int(label),
      'x_Center': x_center,
      'y_Center': y_center,
      'width': width,
      'height': height,
      'x_min': x_min,
      'y_min': y_min,
      'x_max': x_max,
      'y_max': y_max,
      #'path' : f"{folder_train_img}/"
    })

  file.close

In [ ]:
df_train = pd.DataFrame(data_train)
df_train.head()

,Filename,Label,x_Center,y_Center,width,height,x_min,y_min,x_max,y_max
0,volume_191_slice_88_jpg.rf.cb43fa83061673bfbb9...,0,0.715827,0.318182,0.438849,0.393939,69.0,16.0,130.0,68.0
1,volume_191_slice_88_jpg.rf.cb43fa83061673bfbb9...,1,0.672662,0.337121,0.640288,0.553030,49.0,8.0,138.0,81.0
2,volume_191_slice_88_jpg.rf.cb43fa83061673bfbb9...,2,0.726619,0.325758,0.532374,0.424242,64.0,15.0,138.0,71.0
3,volume_171_slice_132_jpg.rf.b00c31315090d71226...,1,0.460432,0.428030,0.172662,0.113636,52.0,49.0,76.0,64.0
4,volume_280_slice_97_jpg.rf.f1912158a7c5f0f4161...,0,0.420863,0.829545,0.050360,0.037879,55.0,107.0,62.0,112.0


In [ ]:
df_train.to_csv('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/train.csv')

In [ ]:
df_train.shape

(15041, 10)

## Validation dataframe

In [ ]:
for labels_file in os.listdir(folder_val_label):
  file_path = f"{folder_val_label}/{labels_file}"
  file = open(file_path,"r")
  content = file.readlines()
  filename = labels_file[:-4] + '.jpg'

  for line in content:
    parts = line.strip().split()
    label = int(parts[0])
    x_center = float(parts[1])
    y_center = float(parts[2])
    width = float(parts[3])
    height = float(parts[4])

    x_min,y_min,x_max,y_max = calculate(folder_val_img,filename,x_center,y_center,width,height)

    data_val.append({
      'Filename': filename,
      'Label': int(label),
      'x_Center': x_center,
      'y_Center': y_center,
      'width': width,
      'height': height,
      'x_min': x_min,
      'y_min': y_min,
      'x_max': x_max,
      'y_max': y_max,
      #'path' : f"{folder_val_img}/"
    })

  file.close

In [ ]:
df_val = pd.DataFrame(data_val)
df_val.head()

,Filename,Label,x_Center,y_Center,width,height,x_min,y_min,x_max,y_max
0,volume_251_slice_77_jpg.rf.80f3066dd4d805760af...,0,0.359712,0.818182,0.230216,0.212121,34.0,94.0,66.0,122.0
1,volume_251_slice_77_jpg.rf.80f3066dd4d805760af...,1,0.399281,0.791667,0.424460,0.325758,26.0,83.0,85.0,126.0
2,volume_251_slice_77_jpg.rf.80f3066dd4d805760af...,2,0.352518,0.829545,0.244604,0.234848,32.0,94.0,66.0,125.0
3,volume_156_slice_52_jpg.rf.cf02768007e6460bc86...,0,0.359712,0.700758,0.330935,0.250000,27.0,76.0,73.0,109.0
4,volume_156_slice_52_jpg.rf.cf02768007e6460bc86...,1,0.467626,0.731061,0.546763,0.386364,27.0,71.0,103.0,122.0


In [ ]:
df_val.to_csv('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/valid/valid.csv')

# Preprocess

## Augmentation

Augmentation base on the label of interest

In [ ]:
import cv2
import os
import albumentations as A
import pandas as pd

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.7),
    A.RandomBrightnessContrast(p=0.7),
    A.Rotate(limit=40, p=0.7),
    A.RandomCrop(width=128, height=128, p=1.0),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

# Function to convert bounding boxes from Pascal VOC format to YOLO format
def convert_to_yolo(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    center_x = (x_min + x_max) / 2 / img_width
    center_y = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return [center_x, center_y, width, height]

# Function to load an image, apply augmentation, and save the augmented image and bounding boxes
def augment_and_save(img_name, image_path, save_img, save_label, bboxes, class_labels, augmentations, num_augmentations=1):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image is loaded properly
    if image is None:
        raise FileNotFoundError(f"Image at path {image_path} could not be loaded. Please check if the file exists and the path is correct.")

    img_height, img_width = image.shape[:2]

    # Convert bounding boxes to YOLO format
    yolo_bboxes = [convert_to_yolo(bbox, img_width, img_height) for bbox in bboxes]

    # Create save directory if it doesn't exist
    if not os.path.exists(save_img):
        os.makedirs(save_img)

    if not os.path.exists(save_label):
        os.makedirs(save_label)



    '''if not os.path.exists(save_label):
        os.makedirs(save_label)'''

    # Perform augmentations multiple times
    for i in range(num_augmentations):
        augmented = augmentations(image=image, bboxes=yolo_bboxes, class_labels=class_labels)
        augmented_image = augmented["image"]
        augmented_bboxes = augmented["bboxes"]

        # Save the augmented image
        save_image_path = os.path.join(save_img, f"{img_name[:-4]}_augmented_{i}.jpg")
        cv2.imwrite(save_image_path, augmented_image)

        # Save the augmented bounding boxes in YOLO format
        save_label_path = os.path.join(save_label, f"{img_name[:-4]}_augmented_{i}.txt")
        with open(save_label_path, 'w') as f:
            for bbox, label in zip(augmented_bboxes, class_labels):
                center_x, center_y, width, height = bbox
                f.write(f"{label} {center_x} {center_y} {width} {height}\n")

        print(f"Saved: {save_image_path} and {save_label_path}")

In [ ]:
num_label1 = (df_train['Label'] == 1).value_counts()
print(num_label1)

Label
False    8296
True     6745
Name: count, dtype: int64


In [ ]:
label_1_count = 0
for img_name in os.listdir('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images/'):
    image_path = f'/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images/{img_name}'
    save_img = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images/'
    save_label = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels/'

    # Filter the bounding boxes from the dataframe for the current image
    bboxes = []
    class_labels = []

    for _, row in df_train[df_train['Filename'] == img_name].iterrows():
        label = row['Label']
        x_min = row['x_min']
        y_min = row['y_min']
        x_max = row['x_max']
        y_max = row['y_max']
        bboxes.append([x_min, y_min, x_max, y_max])
        class_labels.append(label)

    # Apply augmentations and save images and labels
    for i in range(len(class_labels)):
      if class_labels[i]==1:
          label_1_count += 1
          if label_1_count == (num_label1 // 2).iloc[1]:
              augment_and_save(img_name, image_path, save_img, save_label, bboxes, class_labels, transform, num_augmentations=3)
      else:
          augment_and_save(img_name, image_path, save_img, save_label, bboxes, class_labels, transform, num_augmentations=3)

Streaming output truncated to the last 5000 lines.
Saved: /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images/volume_369_slice_132_jpg.rf.bb557c13bb3b27def3173b0984318994_augmented_1.jpg and /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels/volume_369_slice_132_jpg.rf.bb557c13bb3b27def3173b0984318994_augmented_1.txt
Saved: /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images/volume_369_slice_132_jpg.rf.bb557c13bb3b27def3173b0984318994_augmented_2.jpg and /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels/volume_369_slice_132_jpg.rf.bb557c13bb3b27def3173b0984318994_augmented_2.txt
Saved: /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images/volume_45_slice_58_jpg.rf.9396ee3af457f1dc13b69ad696ffcd19_augmented_0.jpg and /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels/volume_45_slice_58_jpg.rf.9396ee3af457f1dc13b69ad696ffcd19_

## Create df_aug

In [ ]:
import os
from os import listdir

folder_aug_img = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images/"
folder_aug_label = "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels/"

In [ ]:
df_aug = []

In [ ]:
# Calculate xmin,ymin,xmax,ymax

def calculate(filename,x_center,y_center,width,height):
  path_img = f"{folder_aug_img}/{filename}"

  im = Image.open(path_img).convert("RGB")
  img_width, img_height = im.size

  x_center_pixel = x_center * img_width
  y_center_pixel = y_center * img_height
  half_width_pixel = width * img_width / 2
  half_height_pixel = height * img_height / 2

  x_min = int(x_center_pixel - half_width_pixel)
  y_min = int(y_center_pixel - half_height_pixel)
  x_max = int(x_center_pixel + half_width_pixel)
  y_max = int(y_center_pixel + half_height_pixel)

  return x_min,y_min,x_max,y_max

In [ ]:
for labels_file in os.listdir(folder_aug_label):
  file_path = f"{folder_aug_label}/{labels_file}"

  file = open(file_path,"r")
  content = file.readlines()
  filename = labels_file[:-4] + '.jpg'

  for line in content:
    parts = line.strip().split()
    label = int(parts[0])
    x_center = float(parts[1])
    y_center = float(parts[2])
    width = float(parts[3])
    height = float(parts[4])

    x_min,y_min,x_max,y_max = calculate(filename,x_center,y_center,width,height)

    df_aug.append({
      'Filename': filename,
      'Label': int(label),
      'x_Center': x_center,
      'y_Center': y_center,
      'width': width,
      'height': height,
      'x_min': x_min,
      'y_min': y_min,
      'x_max': x_max,
      'y_max': y_max,
        #'path' : f"{folder_train_img}/"
    })

  file.close

In [ ]:
df_aug = pd.DataFrame(df_aug)
df_aug.head()

,Filename,Label,x_Center,y_Center,width,height,x_min,y_min,x_max,y_max
0,volume_279_slice_78_jpg.rf.9ba73757b5b2767c476...,0,0.226562,0.246094,0.453125,0.445312,0,3,57,60
1,volume_279_slice_78_jpg.rf.9ba73757b5b2767c476...,1,0.234375,0.285156,0.468750,0.414062,0,10,60,63
2,volume_156_slice_69_jpg.rf.60b14796fea53d2f8a4...,0,0.311657,0.636228,0.416897,0.402363,13,55,66,107
3,volume_156_slice_69_jpg.rf.60b14796fea53d2f8a4...,1,0.413973,0.671982,0.784315,0.656036,2,44,103,128
4,volume_156_slice_69_jpg.rf.60b14796fea53d2f8a4...,2,0.338607,0.630172,0.397760,0.378381,17,56,68,104


In [ ]:
df_aug.to_csv('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_train.csv')

In [ ]:
df_aug.shape

(38680, 10)

In [ ]:
df_aug['Label'].value_counts()

,count
Label,
1,13830
0,12714
2,12136


#delete folder images in train and rename folder augmented_images to images

In [ ]:
import os, shutil
folder_img = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images'
folder_label = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/labels'

for filename in os.listdir(folder_img):
    file_path = os.path.join(folder_img, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

for filename in os.listdir(folder_label):
    file_path = os.path.join(folder_label, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
os.rename("/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_images", "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images")
os.rename("/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/augmented_labels", "/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/labels")

# Load YOLOv8 Pre-trained Model

In [ ]:
# Load a pretrained YOLOv8n model from Ultralytics
T_Model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:00<00:00, 258MB/s]


# Train Model

In [ ]:
# Define the dataset_path
yaml_file_path = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/data.yaml'

# Train the model on our custom dataset
results = T_Model.train( data= yaml_file_path,epochs=5,patience=20, batch= 16, optimizer='auto')

Ultralytics YOLOv8.2.94 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/data.yaml, epochs=5, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/labels.cache... 14160 images, 1 backgrounds, 0 corrupt: 100%|██████████| 14160/14160 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/TumorDetectionYolov8/OD8/Brain Tumor Detection/valid/labels.cache... 1980 images, 17 backgrounds, 0 corrupt: 100%|██████████| 1980/1980 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1980       4380      0.517      0.482      0.451      0.193



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        4/5      10.5G      1.529      1.837      1.869         91        640: 100%|██████████| 885/885 [12:27<00:00,  1.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:40<00:00,  1.54it/s]


                   all       1980       4380      0.653      0.515      0.554      0.269

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      10.3G      1.472      1.665       1.83         66        640:  66%|██████▌   | 584/885 [08:12<04:24,  1.14it/s]

# Evaluation Test

In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('/content/runs/detect/train2/weights/best.pt')

# Path to the test images directory
test_images_dir = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/test/images'

# Perform detection on test images and save results
results = model.predict(source=test_images_dir, save=True, save_txt=True)

In [ ]:
img = Image.open('/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/test/images/volume_336_slice_106_jpg.rf.9672b03cda1146a8dbd6cece7f62e2f1.jpg')
plt.imshow(img)

In [ ]:
img_pred = Image.open('/content/runs/detect/predict3/volume_336_slice_106_jpg.rf.9672b03cda1146a8dbd6cece7f62e2f1.jpg')
plt.imshow(img_pred)

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

path = '/content/runs/detect/predict3/'
images_shown = 0

plt.figure(figsize=(15, 10))  # Adjust figure size as needed

for i in os.listdir(path):
    item_path = path + i
    if os.path.isfile(item_path) and item_path.lower().endswith('.jpg'):  # Check if it's an image file
        img = Image.open(item_path)

        plt.subplot(3, 3, images_shown + 1)  # 3 rows, 3 columns of images
        plt.imshow(img)
        plt.axis('off')  # Turn off axis labels

        images_shown += 1
        if images_shown >= 9:
            break

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
index_img = []
for i in range(len(df_aug)):
  if df_aug['Filename'][i] == 'volume_100_slice_70_jpg.rf.107f05d1e8d46e2122959586bdabbe9a.jpg':
    index_img.append(i)

print(index_img)

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

for i in index_img:
  path_img = '/content/TumorDetectionYolov8/OD8/Brain Tumor Detection/train/images/'
  im = Image.open(path_img + df_aug['Filename'][i]).convert('RGB')
  print(im.size)
  print(df_aug['Label'][i])
  print(df_aug['x_min'][i])
  print(df_aug['y_min'][i])
  print(df_aug['x_max'][i])
  print(df_aug['y_max'][i])
  print(df_aug['Filename'][i])
  print(df_aug['path'][i])
  draw = ImageDraw.Draw(im)
  boxs = df_aug[['x_min','y_min','x_max','y_max']].iloc[i]
  draw.rectangle(boxs,width=1,outline="red")
  plt.imshow(im)
  plt.show() # Added to display each image individually

# Model performent evaluation

In [ ]:
display.Image('/content/runs/detect/train2/results.png')

In [ ]:
display.Image('/content/runs/detect/train2/confusion_matrix_normalized.png')